In [28]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [29]:
# Subclassing
# CNN -> BatchNorm -> ReLU (common structure)
# x10 (a lot of code to write!)

class CNNBlock(layers.Layer):
  def __init__(self, out_channels, kernel_size=3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
    self.bn = layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv(input_tensor)
    x = self.bn(x, training=training)
    x = layers.Activation("relu")(x)
    return x

In [30]:
class ResBlock(layers.Layer):
  def __init__(self, channels):
    super(ResBlock, self).__init__()
    self.cnn1 = CNNBlock(channels[0],3)
    self.cnn2 = CNNBlock(channels[1], 3)
    self.cnn3 = CNNBlock(channels[2], 3)
    self.identity_mapping = layers.Conv2D(channels[1], 1, padding="same")
    self.pooling = layers.MaxPool2D()

  def call(self,input_tensor, training=False):
    x = self.cnn1(input_tensor, training=training)
    x = self.cnn2(x, training=training)
    x = self.cnn3(x + self.identity_mapping(input_tensor), training=training)
    x = self.pooling(x)
    return x

In [31]:
class ResNetLike(keras.Model):
  def __init__(self,num_classes=10):
    super(ResNetLike, self).__init__()
    self.block1 = ResBlock([32, 32, 64])
    self.block2 = ResBlock([128, 128, 256])
    self.block3 = ResBlock([128, 256, 512])
    self.pool = layers.GlobalAveragePooling2D()
    self.classifier = layers.Dense(num_classes)

  def call(self, input_tensor, training=False):
    x = self.block1(input_tensor, training=training)
    x = self.block2(x, training=training)
    x = self.block3(x, training=training)
    x = self.pool(x, training=training)
    x = self.classifier(x)
    return x

  def model(self):
    x = keras.Input(shape=(28, 28, 1))
    return keras.Model(inputs=[x], outputs=self.call(x))

In [32]:
model = ResNetLike(10)
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=True)
# print(model.summary())
model.evaluate(x_test, y_test, batch_size=64, verbose=True)

157/157 [==============================] - 3s 16ms/step - loss: 0.1152 - accuracy: 0.9670


[0.11522364616394043, 0.9670000076293945]

In [34]:
model.model().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block_6 (ResBlock)       (None, 14, 14, 64)        28640     
_________________________________________________________________
res_block_7 (ResBlock)       (None, 7, 7, 256)         526976    
_________________________________________________________________
res_block_8 (ResBlock)       (None, 3, 3, 512)         1839744   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 2,400,490
Trainable params: 2,397,418
Non-trainable params: 3,072
_______________________________________________